This is all just for fun. The author is not a racist in any way.

In [22]:
from sklearn.model_selection import train_test_split
import torch
import pandas as pd

In [23]:
train_data_unintended_bias = pd.read_csv("./Data/jigsaw-unintended-bias-train.csv")

In [24]:
train_data_unintended_bias.dropna(inplace = True)

train_data_unintended_bias["sum"] = train_data_unintended_bias.iloc[:, 8:32].values.sum(axis = 1)
train_data_unintended_bias = train_data_unintended_bias[train_data_unintended_bias["sum"] >= 0.5]

x_test = torch.tensor(train_data_unintended_bias.iloc[:, 8:32].values, dtype = torch.float)
y_test = torch.tensor(train_data_unintended_bias.iloc[:, 2].values, dtype = torch.float)

print("toxicity fraction in test: ", y_test.round().sum() / y_test.shape[0])

toxic = train_data_unintended_bias[train_data_unintended_bias["toxic"] >= 0.5]
untoxic = train_data_unintended_bias[train_data_unintended_bias["toxic"] < 0.5].sample(10000)

data = pd.concat((toxic, toxic, untoxic))
print("toxicity fraction in train: ", data.iloc[:, 2].values.round().sum() / data.shape[0])

toxicity fraction in test dataset:  tensor(0.1008)
toxicity fraction in test dataset:  0.513152514651765


In [4]:
racist_x_train = torch.tensor(data.iloc[:, 8:32].values, dtype = torch.float)
racist_y_train = torch.tensor(data.iloc[:, 2].values, dtype = torch.float)

def make_labels(x):
    for i in range(x.shape[0]):
        x[i] = 0 if x[i] <= 0.5 else 1
    return x
        
racist_y_train = make_labels(racist_y_train)
y_test = make_labels(y_test)     

In [5]:
racist_x_train, racist_x_test, racist_y_train, racist_y_test = train_test_split(
    racist_x_train, 
    racist_y_train, 
    test_size = 0.3,
    shuffle = True)

In [6]:
class CatDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        
        self.x = x
        self.y = y
        self.index_list = []
        
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, i):
        return (self.x[i], self.y[i])

In [7]:
import torch.nn as nn

In [8]:
class RacistClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear(24, 64)
        self.softmax = nn.Softmax(dim = 1)
        self.elu = nn.ELU()
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = self.linear1(x.float())
        x = self.elu(x)
        x = self.linear2(x)
        x = self.elu(x)
        x = self.linear3(x)
        
        return x
    
    def predict(self, x):
        return self.softmax(self.forward(x))

In [9]:
model = RacistClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)
cross_entropy = nn.CrossEntropyLoss()

In [10]:
training_set = CatDataset(racist_x_train, racist_y_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size = 512, shuffle = True)

validation_set = CatDataset(racist_x_test, racist_y_test)
validation_generator = torch.utils.data.DataLoader(validation_set)

n_iterations = 500

from sklearn.metrics import accuracy_score
from tqdm import tqdm

loss = 0

for epoch in tqdm(range(n_iterations)):
    for local_batch, local_labels in training_generator:
        optimizer.zero_grad()
        output = model(local_batch)

        loss = cross_entropy(output, local_labels.long())

        loss.backward()
        optimizer.step()

100%|██████████| 500/500 [07:43<00:00,  1.08it/s]


In [11]:
from sklearn.metrics import accuracy_score

# the first score is on train class distribution
# the second score is on initial class distribution
print(accuracy_score(torch.argmax(model.predict(racist_x_test), dim = 1), racist_y_test))
accuracy_score(torch.argmax(model.predict(x_test), dim = 1), y_test)

0.5819641099416976


0.5593630118654652

In [12]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth = 20)
random_forest.fit(racist_x_train, racist_y_train)

print(random_forest.score(racist_x_test, racist_y_test))
random_forest.score(x_test, y_test)

0.6299689558567426


0.5584173432063521

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

boosting = GradientBoostingClassifier()
boosting.fit(racist_x_train, racist_y_train)

print(boosting.score(racist_x_test, racist_y_test))
boosting.score(x_test, y_test)

0.5817369576739608


0.5311803015434027